In [161]:
import pandas as pd
import json
from pprint import pprint
from util.managers.DataframeManager import DataframeManager
from util.managers.DataframeManager import ColumnName
from util.managers.CountryCodeManager import CountryCodeManager
from util.managers.UnitConverterManager import UnitConverterManager

# Variables used across the notebook and not changed

In [162]:
DATASET_ID = "T006"
dataframeManager = DataframeManager(DATASET_ID)
countryCodeManager = CountryCodeManager()

# Opening the dataset

In [163]:
filename = "modal-split-of-freight-transport.csv"
path = "/Users/hlinero/Desktop/iTEM Material/raw dataset/T006/{}".format(filename)
df = dataframeManager.get_dataframe_from_csv_file(path, ";")
df

,Date,Frequency,Geo,Measure,Tra Mode,Value
0,2012,Annual,Czech Republic,Percentage,"Railways, inland waterways - sum of available ...",30.6
1,2008,Annual,Croatia,Percentage,Roads,71.7
2,2009,Annual,Croatia,Percentage,Roads,71.8
3,2017,Annual,Belgium,Percentage,"Railways, inland waterways - sum of available ...",26.3
4,2009,Annual,Greece,Percentage,"Railways, inland waterways - sum of available ...",2.1
...,...,...,...,...,...,...
1401,2009,Annual,Norway,Percentage,Roads,83.6
1402,2011,Annual,Norway,Percentage,Roads,84.2
1403,2006,Annual,Switzerland,Percentage,Roads,65.0
1404,2008,Annual,Switzerland,Percentage,Inland waterways,0.1


# Dropping unnecessary columns

In [164]:
df.drop(columns=["Frequency", "Measure" ], inplace = True)
df

,Date,Geo,Tra Mode,Value
0,2012,Czech Republic,"Railways, inland waterways - sum of available ...",30.6
1,2008,Croatia,Roads,71.7
2,2009,Croatia,Roads,71.8
3,2017,Belgium,"Railways, inland waterways - sum of available ...",26.3
4,2009,Greece,"Railways, inland waterways - sum of available ...",2.1
...,...,...,...,...
1401,2009,Norway,Roads,83.6
1402,2011,Norway,Roads,84.2
1403,2006,Switzerland,Roads,65.0
1404,2008,Switzerland,Inland waterways,0.1


# Drop unnecessary rows
    Rule: We will erase all rows that represent a 'Tra Mode' of 'Railways, inland waterways - sum of available data'

In [165]:
# Variable holding the index of the rows to be erased
rows_to_erase = []

# Getting all the indeces to erase
for index, row in df.iterrows():
    mode = row["Tra Mode"]
    if mode == 'Railways, inland waterways - sum of available data':
        rows_to_erase.append(index)

# Erasing the undesired rows
df_len_before_drop = len(df)
df.drop(df.index[rows_to_erase], inplace = True)
df.reset_index(inplace=True)
df.drop(columns=["index" ], inplace = True)

# Asserting that the new df has less rows than before the erase
df

,Date,Geo,Tra Mode,Value
0,2008,Croatia,Roads,71.7
1,2009,Croatia,Roads,71.8
2,2005,Estonia,Roads,20.0
3,2011,Estonia,Roads,28.4
4,2012,Estonia,Roads,33.1
...,...,...,...,...
1024,2009,Norway,Roads,83.6
1025,2011,Norway,Roads,84.2
1026,2006,Switzerland,Roads,65.0
1027,2008,Switzerland,Inland waterways,0.1


# Renaming the "Geo" column to "Country"
    Rule: Renaming the column to comply with the template

In [166]:
dataframeManager.rename_column(current_name="Geo",df=df,new_name=ColumnName.COUNTRY.value)
df

,Date,Country,Tra Mode,Value
0,2008,Croatia,Roads,71.7
1,2009,Croatia,Roads,71.8
2,2005,Estonia,Roads,20.0
3,2011,Estonia,Roads,28.4
4,2012,Estonia,Roads,33.1
...,...,...,...,...
1024,2009,Norway,Roads,83.6
1025,2011,Norway,Roads,84.2
1026,2006,Switzerland,Roads,65.0
1027,2008,Switzerland,Inland waterways,0.1


# Renaming the "Date" column to "Year"
    Rule: Renaming the column to comply with the template

In [167]:
dataframeManager.rename_column(current_name="Date",df=df,new_name=ColumnName.YEAR.value)
df

,Year,Country,Tra Mode,Value
0,2008,Croatia,Roads,71.7
1,2009,Croatia,Roads,71.8
2,2005,Estonia,Roads,20.0
3,2011,Estonia,Roads,28.4
4,2012,Estonia,Roads,33.1
...,...,...,...,...
1024,2009,Norway,Roads,83.6
1025,2011,Norway,Roads,84.2
1026,2006,Switzerland,Roads,65.0
1027,2008,Switzerland,Inland waterways,0.1


# Adding the "Source" column
    Rule: This dataset comes from the Eurostat

In [168]:
dataframeManager.simple_column_insert(cell_value="Eurostat", dataframe=df, column_name= ColumnName.SOURCE.value)
df

,Source,Year,Country,Tra Mode,Value
0,Eurostat,2008,Croatia,Roads,71.7
1,Eurostat,2009,Croatia,Roads,71.8
2,Eurostat,2005,Estonia,Roads,20.0
3,Eurostat,2011,Estonia,Roads,28.4
4,Eurostat,2012,Estonia,Roads,33.1
...,...,...,...,...,...
1024,Eurostat,2009,Norway,Roads,83.6
1025,Eurostat,2011,Norway,Roads,84.2
1026,Eurostat,2006,Switzerland,Roads,65.0
1027,Eurostat,2008,Switzerland,Inland waterways,0.1


# Getting the ISO code for each country
    Rule: For each country we need its ISO code

# Determining which countries do not have an ISO code
    Rule: As shown below, there are two countries which do not have an ISO code. The mapping that will be done in order to handle those cases are as follows:
       > Germany (until 1990 former territory of the FRG) ----> Germany
       > European Union (current composition) ----> EU28

In [169]:
list_of_countries = list(set(df[ColumnName.COUNTRY.value]))
countries_with_no_iso_code = countryCodeManager.get_list_of_countries_with_no_iso_code(list_of_countries)
countries_with_no_iso_code

['Germany (until 1990 former territory of the FRG)',
 'European Union (current composition)']

# Cleaning the list of countries to obtain their ISO code

In [170]:
country_column = list(df[ColumnName.COUNTRY.value])
list_of_iso_code = []

for country in country_column:
    if country == "Germany (until 1990 former territory of the FRG)":
        code = countryCodeManager.get_iso_code_for_country("Germany")
    elif country == "European Union (current composition)":
        code = "EU28"
    else:
        code = countryCodeManager.get_iso_code_for_country(country)

    # Appending the code to the list
    list_of_iso_code.append(code)
        
# Assert that the size of the list of iso codes is equivalent to the size of country column
assert len(list_of_iso_code) == len(country_column)
    
# Adding the ISO column to the dataframe
df.insert(2, ColumnName.ISO_CODE.value, list_of_iso_code, True)
df        

,Source,Year,ISO Code,Country,Tra Mode,Value
0,Eurostat,2008,HRV,Croatia,Roads,71.7
1,Eurostat,2009,HRV,Croatia,Roads,71.8
2,Eurostat,2005,EST,Estonia,Roads,20.0
3,Eurostat,2011,EST,Estonia,Roads,28.4
4,Eurostat,2012,EST,Estonia,Roads,33.1
...,...,...,...,...,...,...
1024,Eurostat,2009,NOR,Norway,Roads,83.6
1025,Eurostat,2011,NOR,Norway,Roads,84.2
1026,Eurostat,2006,CHE,Switzerland,Roads,65.0
1027,Eurostat,2008,CHE,Switzerland,Inland waterways,0.1


# Getting the ITEM code for each ISO region
    Rule: For each coutry, we need to assign an ITEM region

# Determining which counties do not have an ITEM region
    Rule: As seen below, the only country that has no region is the one with an ISO of EU28. Therefore, what we will do is assign the region 'EU-28' to the row that has such an ISO.

In [171]:
list_of_unique_iso_codes = list(set(df[ColumnName.ISO_CODE.value]))
iso_codes_with_no_region = countryCodeManager.get_list_of_iso_codes_with_no_region(list_of_unique_iso_codes)
iso_codes_with_no_region

['EU28']

# Assigning the ITEM region column

In [172]:
# Getting the complete list of iso codes
list_of_all_codes = df[ColumnName.ISO_CODE.value]

# Getting the list of regions
item_regions = countryCodeManager.get_list_of_regions_for_iso_codes(list_of_all_codes)

# Hardcoding the region value for WLD ISO Code
for index in range(0, len(list_of_all_codes)):
    if list_of_all_codes[index] == "N/A":
        item_regions[index] = "EU-28"

# Adding the column to the dataframe
df.insert(3, ColumnName.ITEM_REGION.value, item_regions, True)
df

,Source,Year,ISO Code,Region,Country,Tra Mode,Value
0,Eurostat,2008,HRV,EU-27,Croatia,Roads,71.7
1,Eurostat,2009,HRV,EU-27,Croatia,Roads,71.8
2,Eurostat,2005,EST,EU-27,Estonia,Roads,20.0
3,Eurostat,2011,EST,EU-27,Estonia,Roads,28.4
4,Eurostat,2012,EST,EU-27,Estonia,Roads,33.1
...,...,...,...,...,...,...,...
1024,Eurostat,2009,NOR,Non-EU Europe,Norway,Roads,83.6
1025,Eurostat,2011,NOR,Non-EU Europe,Norway,Roads,84.2
1026,Eurostat,2006,CHE,Non-EU Europe,Switzerland,Roads,65.0
1027,Eurostat,2008,CHE,Non-EU Europe,Switzerland,Inland waterways,0.1


# Adding the 'Variable' and 'Unit' columns
    Rule: This dataset is associated to freight. Therefore, the variable is 'Freight Activity' and the unit is '% tonne-kilometres / yr'

## Variable column

In [173]:
dataframeManager.simple_column_insert(df, ColumnName.VARIABLE.value, "Freight Activity", 4)
df

,Source,Year,ISO Code,Region,Variable,Country,Tra Mode,Value
0,Eurostat,2008,HRV,EU-27,Freight Activity,Croatia,Roads,71.7
1,Eurostat,2009,HRV,EU-27,Freight Activity,Croatia,Roads,71.8
2,Eurostat,2005,EST,EU-27,Freight Activity,Estonia,Roads,20.0
3,Eurostat,2011,EST,EU-27,Freight Activity,Estonia,Roads,28.4
4,Eurostat,2012,EST,EU-27,Freight Activity,Estonia,Roads,33.1
...,...,...,...,...,...,...,...,...
1024,Eurostat,2009,NOR,Non-EU Europe,Freight Activity,Norway,Roads,83.6
1025,Eurostat,2011,NOR,Non-EU Europe,Freight Activity,Norway,Roads,84.2
1026,Eurostat,2006,CHE,Non-EU Europe,Freight Activity,Switzerland,Roads,65.0
1027,Eurostat,2008,CHE,Non-EU Europe,Freight Activity,Switzerland,Inland waterways,0.1


## Unit Column

In [174]:
dataframeManager.simple_column_insert(df, ColumnName.UNIT.value, "% tonne-kilometres / yr", 5)
df

,Source,Year,ISO Code,Region,Variable,Unit,Country,Tra Mode,Value
0,Eurostat,2008,HRV,EU-27,Freight Activity,% tonne-kilometres / yr,Croatia,Roads,71.7
1,Eurostat,2009,HRV,EU-27,Freight Activity,% tonne-kilometres / yr,Croatia,Roads,71.8
2,Eurostat,2005,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Estonia,Roads,20.0
3,Eurostat,2011,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Estonia,Roads,28.4
4,Eurostat,2012,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Estonia,Roads,33.1
...,...,...,...,...,...,...,...,...,...
1024,Eurostat,2009,NOR,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Norway,Roads,83.6
1025,Eurostat,2011,NOR,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Norway,Roads,84.2
1026,Eurostat,2006,CHE,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Switzerland,Roads,65.0
1027,Eurostat,2008,CHE,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Switzerland,Inland waterways,0.1


# Adding the 'Service' Column
    Rule: Since all the variable is 'Freight Activity' the service is 'Freight'

In [175]:
dataframeManager.simple_column_insert(df, ColumnName.SERVICE.value, "Freight", 6)
df

,Source,Year,ISO Code,Region,Variable,Unit,Service,Country,Tra Mode,Value
0,Eurostat,2008,HRV,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Croatia,Roads,71.7
1,Eurostat,2009,HRV,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Croatia,Roads,71.8
2,Eurostat,2005,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Estonia,Roads,20.0
3,Eurostat,2011,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Estonia,Roads,28.4
4,Eurostat,2012,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Estonia,Roads,33.1
...,...,...,...,...,...,...,...,...,...,...
1024,Eurostat,2009,NOR,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,Norway,Roads,83.6
1025,Eurostat,2011,NOR,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,Norway,Roads,84.2
1026,Eurostat,2006,CHE,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,Switzerland,Roads,65.0
1027,Eurostat,2008,CHE,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,Switzerland,Inland waterways,0.1


# Adding the 'Technology', 'Fuel', 'Vehicle type' columns
    Rule: Since the dataset does not provide any information about 'technology' and 'fuel,' we set both values as 'All'

In [176]:
dataframeManager.simple_column_insert(df, ColumnName.TECHNOLOGY.value, "All", 7)
dataframeManager.simple_column_insert(df, ColumnName.FUEL.value, "All", 8)
df

,Source,Year,ISO Code,Region,Variable,Unit,Service,Technology,Fuel,Country,Tra Mode,Value
0,Eurostat,2008,HRV,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,All,All,Croatia,Roads,71.7
1,Eurostat,2009,HRV,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,All,All,Croatia,Roads,71.8
2,Eurostat,2005,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,All,All,Estonia,Roads,20.0
3,Eurostat,2011,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,All,All,Estonia,Roads,28.4
4,Eurostat,2012,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,All,All,Estonia,Roads,33.1
...,...,...,...,...,...,...,...,...,...,...,...,...
1024,Eurostat,2009,NOR,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,All,All,Norway,Roads,83.6
1025,Eurostat,2011,NOR,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,All,All,Norway,Roads,84.2
1026,Eurostat,2006,CHE,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,All,All,Switzerland,Roads,65.0
1027,Eurostat,2008,CHE,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,All,All,Switzerland,Inland waterways,0.1


# Setting the 'Mode' and 'Vehicle Type' Columns based on the given data
    Rule: The following is the mapping done for setting the mode:
    Tra Mode --> Mode --> Vehicle Type
    Railways -> Rail --> All
    Roads -> Road --> All
    Inland waterways --> Shipping -> Inland Waterway

In [177]:
# variables holding the mode and vehicle type
result_mode = []
result_vehicle_type = []

# Getting the mode and vehicle type of each row
for index, row in df.iterrows():
    tra_mode = row['Tra Mode']
    if tra_mode == 'Railways':
        result_mode.append("Rail")
        result_vehicle_type.append("All")
    elif tra_mode == 'Roads':
        result_mode.append("Road")
        result_vehicle_type.append("All")
    else:
        result_mode.append("Shipping")
        result_vehicle_type.append("Inland Waterway")

# Asserting that the len of the results
assert len(df) == len(result_mode)
assert len(df) == len(result_vehicle_type)

# Adding the columns to the dataframe
df.insert(7, ColumnName.MODE.value, result_mode, True)
df.insert(8, ColumnName.VEHICLE_TYPE.value, result_vehicle_type, True)

# Removing the 'Tra Mode' column
df.drop(columns=["Tra Mode" ], inplace = True)

df

,Source,Year,ISO Code,Region,Variable,Unit,Service,Mode,Vehicle Type,Technology,Fuel,Country,Value
0,Eurostat,2008,HRV,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,Croatia,71.7
1,Eurostat,2009,HRV,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,Croatia,71.8
2,Eurostat,2005,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,Estonia,20.0
3,Eurostat,2011,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,Estonia,28.4
4,Eurostat,2012,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,Estonia,33.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,Eurostat,2009,NOR,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,Norway,83.6
1025,Eurostat,2011,NOR,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,Norway,84.2
1026,Eurostat,2006,CHE,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,Switzerland,65.0
1027,Eurostat,2008,CHE,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,Shipping,Inland Waterway,All,All,Switzerland,0.1


# Reordering the columns
    Rule: To comply with the current template, the columns must be shown in a specific order. The dataframe manager specifies the order of the columns

In [178]:
df = dataframeManager.reorder_columns(df)
df

,Source,Country,ISO Code,Region,Variable,Unit,Service,Mode,Vehicle Type,Technology,Fuel,Value,Year
0,Eurostat,Croatia,HRV,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,71.7,2008
1,Eurostat,Croatia,HRV,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,71.8,2009
2,Eurostat,Estonia,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,20.0,2005
3,Eurostat,Estonia,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,28.4,2011
4,Eurostat,Estonia,EST,EU-27,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,33.1,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,Eurostat,Norway,NOR,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,83.6,2009
1025,Eurostat,Norway,NOR,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,84.2,2011
1026,Eurostat,Switzerland,CHE,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,Road,All,All,All,65.0,2006
1027,Eurostat,Switzerland,CHE,Non-EU Europe,Freight Activity,% tonne-kilometres / yr,Freight,Shipping,Inland Waterway,All,All,0.1,2008


# Exporting Results

In [179]:
# Programming Friendly View
dataframeManager.create_programming_friendly_file(df)

# User Friendly View
dataframeManager.create_user_friendly_file(df)

> PF File saved at: /Users/hlinero/Documents/database/item/historical/scripts
> UF File saved at: /Users/hlinero/Documents/database/item/historical/scripts
